In [1]:
import dearpygui.dearpygui as dpg
import serial
import time

In [2]:
ser = serial.Serial('COM10', 9600, timeout = 1)
ser2 = serial.Serial('COM11', 9600, timeout = 1)
time.sleep(3)

In [3]:
def set_trialphase(button):
    if button == 'Reward-Phase':
        ser.write('r'.encode('utf-8'))
        ser2.write('r'.encode('utf-8'))
    if button == "Intertrial-Phase":
        ser.write('i'.encode('utf-8'))
        ser2.write('i'.encode('utf-8'))

In [4]:
dpg.create_context()
dpg.create_viewport(title='Multiport', width=1000, height=1350, always_on_top=True, x_pos=900, y_pos=30)
dpg.setup_dearpygui()


arduinoData_lp1 = []
arduinoData_lp2 = []
arduinoData_lp3 = []
counterlist = []

labeltable = [[1,2,3,4,5,6,7,8],[9,10,11,12,13,14,15,16]]
labels_trials = [["Reward-Phase","Intertrial-Phase"]]
buttons_trials = {}
buttons_lickports1 = {}
buttons_lickports2 = {}

counter = 0

with dpg.theme() as button_clicked_theme:
    with dpg.theme_component():
        dpg.add_theme_color(dpg.mvThemeCol_Button, (50, 100, 255))
        dpg.add_theme_color(dpg.mvThemeCol_ButtonHovered, (115, 160, 255))
        dpg.add_theme_color(dpg.mvThemeCol_ButtonActive, (100, 190, 255))


def toggle_button_trialphase(sender):
    set_trialphase(dpg.get_item_label(sender))

    for button in buttons_trials.keys():
        if not button == sender:
            buttons_trials[button]["checked"] = False
            dpg.set_value(button, False)
            dpg.bind_item_theme(button, None)
        else:
            buttons_trials[button]["checked"] = True
            dpg.bind_item_theme(button, button_clicked_theme)
            if not dpg.get_value(sender):
                dpg.set_value(sender, True)
            

def toggle_button_lickports(sender):
    
    if sender[6] == "1":
        lib = buttons_lickports1
        ser.write(f"1{sender[8]}".encode("utf-8"))
    else:
        lib = buttons_lickports2
        ser.write(f"2{sender[8]}".encode("utf-8"))

    for button in lib.keys():
        if not button == sender:
            lib[button]["checked"] = False
            dpg.set_value(button, False)
            dpg.bind_item_theme(button, None)
        else:
            lib[button]["checked"] = True
            dpg.bind_item_theme(button, button_clicked_theme)
            if not dpg.get_value(sender):
                dpg.set_value(sender, True)

with dpg.window() as main_window:
    dpg.add_text("Trial Name:", indent= 50)
    dpg.add_input_text(tag = "TrialName")
    dpg.add_text("Trial State", indent = 50)
    with dpg.table(width = 700, header_row=False):
        dpg.add_table_column(label="Header 1")
        dpg.add_table_column(label="Header 2")
        for i in range(1):
            with dpg.table_row():
                for j in range(2):
                    with dpg.table_cell():
                        dpg.add_button(label = labels_trials[i][j], tag=f"button{labels_trials[i][j]}", width = 150, height = 40, callback = toggle_button_trialphase)
                    buttons_trials[f"button{labels_trials[i][j]}"] = {"checked":False}
    dpg.add_text("Reward 1", indent= 50)
    with dpg.table(width = 700, header_row=False):
        dpg.add_table_column(label="Header 1")
        dpg.add_table_column(label="Header 2")
        dpg.add_table_column(label="Header 3")
        dpg.add_table_column(label="Header 4")
        dpg.add_table_column(label="Header 5")
        dpg.add_table_column(label="Header 6")
        dpg.add_table_column(label="Header 7")
        dpg.add_table_column(label="Header 8")
        for i in range(2):
            with dpg.table_row():
                for j in range(8):
                    with dpg.table_cell():
                        dpg.add_button(label = labeltable[i][j], tag=f"button1_{labeltable[i][j]}", width = 100, height = 40, callback = toggle_button_lickports)
                    buttons_lickports1[f"button1_{labeltable[i][j]}"] = {"checked":False}
    dpg.add_text("Reward 2", indent= 50)
    with dpg.table(width = 700, header_row=False):
        dpg.add_table_column(label="Header 1")
        dpg.add_table_column(label="Header 2")
        dpg.add_table_column(label="Header 3")
        dpg.add_table_column(label="Header 4")
        dpg.add_table_column(label="Header 5")
        dpg.add_table_column(label="Header 6")
        dpg.add_table_column(label="Header 7")
        dpg.add_table_column(label="Header 8")
        for i in range(2):
            with dpg.table_row():
                for j in range(8):
                    with dpg.table_cell():
                        dpg.add_button(label = labeltable[i][j], tag=f"button2_{labeltable[i][j]}", width = 100, height = 40, callback = toggle_button_lickports)
                    buttons_lickports2[f"button2_{labeltable[i][j]}"] = {"checked":False}
    with dpg.plot(height=400, width=800):
        dpg.add_plot_axis(dpg.mvXAxis, label="Time", tag="xaxis", no_tick_labels=True)
        dpg.add_plot_axis(dpg.mvYAxis, label="Amplitude", tag="yaxis")
        dpg.add_line_series([], [], tag='line1', parent="yaxis")
    with dpg.plot(height=400, width=800):
        dpg.add_plot_axis(dpg.mvXAxis, label="Time", tag="xaxis1", no_tick_labels=True)
        dpg.add_plot_axis(dpg.mvYAxis, label="Amplitude", tag="yaxis1")
        dpg.add_line_series([], [], tag='line2', parent="yaxis1")
    with dpg.plot(height=400, width=800):
        dpg.add_plot_axis(dpg.mvXAxis, label="Time", tag="xaxis2", no_tick_labels=True)
        dpg.add_plot_axis(dpg.mvYAxis, label="Amplitude", tag="yaxis2")
        dpg.add_line_series([], [], tag='line3', parent="yaxis2")
dpg.set_primary_window(main_window, True)        
dpg.show_viewport()

# below replaces, start_dearpygui()
while dpg.is_dearpygui_running():
    # insert here any code you would like to run in the render loop
    # you can manually stop by using stop_dearpygui()
    newdata1 = ser.readline().decode('utf-8')
    newdata2 = ser.readline().decode('utf-8')
    newdata3 = ser2.readline().decode('utf-8')
    arduinoData_lp1.append(newdata1)
    arduinoData_lp2.append(newdata2)
    arduinoData_lp3.append(newdata3)
    if arduinoData_lp1[-1] == '' or arduinoData_lp1[-1] == '\n' or arduinoData_lp1[-1] == '\r\n':
        arduinoData_lp1[-1] = 1
    else:
        arduinoData_lp1[-1] = int(arduinoData_lp1[-1])
    if arduinoData_lp2[-1] == '' or arduinoData_lp2[-1] == '\n' or arduinoData_lp2[-1] == '\r\n':
        arduinoData_lp2[-1] = 1
    else:
        arduinoData_lp2[-1] = int(arduinoData_lp2[-1])
    if arduinoData_lp3[-1] == '' or arduinoData_lp3[-1] == '\n' or arduinoData_lp3[-1] == '\r\n':
        arduinoData_lp3[-1] = 1
    else:
        arduinoData_lp3[-1] = int(arduinoData_lp3[-1])
    counter +=1
    counterlist.append(counter)
    dpg.configure_item('line1', x=counterlist, y=arduinoData_lp1)
    dpg.configure_item('line2', x=counterlist, y=arduinoData_lp2)
    dpg.configure_item('line3', x=counterlist, y=arduinoData_lp3)
    dpg.fit_axis_data("xaxis")
    dpg.fit_axis_data("yaxis")
    dpg.fit_axis_data("xaxis1")
    dpg.fit_axis_data("yaxis1")
    dpg.fit_axis_data("xaxis2")
    dpg.fit_axis_data("yaxis2")
    dpg.render_dearpygui_frame()

dpg.destroy_context()

In [5]:
# dpg.create_context()
# dpg.create_viewport(title='Multiport', width=1000, height=1350, always_on_top=True, x_pos=900, y_pos=30)
# dpg.setup_dearpygui()


# arduinoData_lp1 = []
# arduinoData_lp2 = []
# counterlist = []

# labeltable = [[1,2,3,4,5,6,7,8],[9,10,11,12,13,14,15,16]]
# labels_trials = [["Reward-Phase","Intertrial-Phase"]]
# buttons_trials = {}
# buttons_lickports = {}

# counter = 0

# with dpg.theme() as button_clicked_theme:
#     with dpg.theme_component():
#         dpg.add_theme_color(dpg.mvThemeCol_Button, (50, 100, 255))
#         dpg.add_theme_color(dpg.mvThemeCol_ButtonHovered, (225, 0, 0))
#         dpg.add_theme_color(dpg.mvThemeCol_ButtonActive, (205, 0, 0))


# def toggle_button_trialphase(sender):
#     set_trialphase(dpg.get_item_label(sender))

#     for button in buttons_trials.keys():
#         if not button == sender:
#             buttons_trials[button]["checked"] = False
#             dpg.set_value(button, False)
#             dpg.bind_item_theme(button, None)
#         else:
#             buttons_trials[button]["checked"] = True
#             dpg.bind_item_theme(button, button_clicked_theme)
#             if not dpg.get_value(sender):
#                 dpg.set_value(sender, True)
            

# def toggle_button_lickports():
#     pass

# with dpg.window() as main_window:
#     dpg.add_text("Trial Name:", indent= 50)
#     dpg.add_input_text(tag = "TrialName")
#     dpg.add_text("Trial State", indent = 50)
#     with dpg.table(width = 700, header_row=False):
#         dpg.add_table_column(label="Header 1")
#         dpg.add_table_column(label="Header 2")
#         for i in range(1):
#             with dpg.table_row():
#                 for j in range(2):
#                     with dpg.table_cell():
#                         dpg.add_button(label = labels_trials[i][j], tag=f"button{labels_trials[i][j]}", width = 150, height = 40, callback = toggle_button_trialphase)
#                     buttons_trials[f"button{labels_trials[i][j]}"] = {"checked":False}
#     dpg.add_text("Reward 1", indent= 50)
#     with dpg.table(width = 700, header_row=False):
#         dpg.add_table_column(label="Header 1")
#         dpg.add_table_column(label="Header 2")
#         dpg.add_table_column(label="Header 3")
#         dpg.add_table_column(label="Header 4")
#         dpg.add_table_column(label="Header 5")
#         dpg.add_table_column(label="Header 6")
#         dpg.add_table_column(label="Header 7")
#         dpg.add_table_column(label="Header 8")
#         for i in range(2):
#             with dpg.table_row():
#                 for j in range(8):
#                     with dpg.table_cell():
#                         dpg.add_button(label = labeltable[i][j], tag=f"button1_{labeltable[i][j]}", width = 100, height = 40, callback = toggle_button_lickports)
#                     buttons_lickports[f"button1_{labeltable[i][j]}"] = {"checked":False}
#     dpg.add_text("Reward 2", indent= 50)
#     with dpg.table(width = 700, header_row=False):
#         dpg.add_table_column(label="Header 1")
#         dpg.add_table_column(label="Header 2")
#         dpg.add_table_column(label="Header 3")
#         dpg.add_table_column(label="Header 4")
#         dpg.add_table_column(label="Header 5")
#         dpg.add_table_column(label="Header 6")
#         dpg.add_table_column(label="Header 7")
#         dpg.add_table_column(label="Header 8")
#         for i in range(2):
#             with dpg.table_row():
#                 for j in range(8):
#                     with dpg.table_cell():
#                         dpg.add_button(label = labeltable[i][j], tag=f"button2_{labeltable[i][j]}", width = 100, height = 40, callback = toggle_button_lickports)
#     with dpg.plot(height=400, width=800):
#         dpg.add_plot_axis(dpg.mvXAxis, label="Time", tag="xaxis", no_tick_labels=True)
#         dpg.add_plot_axis(dpg.mvYAxis, label="Amplitude", tag="yaxis")
#         dpg.add_line_series([], [], tag='line1', parent="yaxis")
#     with dpg.plot(height=400, width=800):
#         dpg.add_plot_axis(dpg.mvXAxis, label="Time", tag="xaxis1", no_tick_labels=True)
#         dpg.add_plot_axis(dpg.mvYAxis, label="Amplitude", tag="yaxis1")
#         dpg.add_line_series([], [], tag='line2', parent="yaxis1")
# dpg.set_primary_window(main_window, True)        
# dpg.show_viewport()

# # below replaces, start_dearpygui()
# while dpg.is_dearpygui_running():
#     # insert here any code you would like to run in the render loop
#     # you can manually stop by using stop_dearpygui()
#     newdata1 = ser.readline().decode('utf-8')
#     newdata2 = ser.readline().decode('utf-8')
#     arduinoData_lp1.append(newdata1)
#     arduinoData_lp2.append(newdata2)
#     if arduinoData_lp1[-1] == '' or arduinoData_lp1[-1] == '\n' or arduinoData_lp1[-1] == '\r\n':
#         arduinoData_lp1[-1] = 1
#     else:
#         arduinoData_lp1[-1] = int(arduinoData_lp1[-1])
#     if arduinoData_lp2[-1] == '' or arduinoData_lp2[-1] == '\n' or arduinoData_lp2[-1] == '\r\n':
#         arduinoData_lp2[-1] = 1
#     else:
#         arduinoData_lp2[-1] = int(arduinoData_lp2[-1])
#     counter +=1
#     counterlist.append(counter)
#     dpg.configure_item('line1', x=counterlist, y=arduinoData_lp1)
#     dpg.configure_item('line2', x=counterlist, y=arduinoData_lp2)
#     dpg.fit_axis_data("xaxis")
#     dpg.fit_axis_data("yaxis")
#     dpg.fit_axis_data("xaxis1")
#     dpg.fit_axis_data("yaxis1")
#     dpg.render_dearpygui_frame()

# dpg.destroy_context()